In [0]:
!pip install -q tensorflow-gpu==2.0.0-rc1

     |████████████████████████████████| 380.5MB 42kB/s 
     |████████████████████████████████| 4.3MB 47.8MB/s 
     |████████████████████████████████| 501kB 48.7MB/s 


In [0]:
!pip install tf-nightly-gpu

In [0]:
# mount to your google drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import numpy as np
import math
import matplotlib.pyplot as plt

print(tf.__version__)

2.2.0-dev20200306


In [0]:
class MobileNet_classification(tf.keras.layers.Layer):
  def __init__(self, pooling='avg', classes=1):
    super(MobileNet_classification, self).__init__()
    self.classes = classes
    self.pooling = pooling

    self.avg_pooling = tf.keras.layers.GlobalAveragePooling2D()
    self.max_pooling = tf.keras.layers.GlobalMaxPooling2D()

    self.sigmoid_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.sigmoid)
    self.softmax_fc = tf.keras.layers.Dense(units=classes, activation=tf.keras.activations.softmax)


  def call(self, inputs):
    if self.pooling == 'avg':
      x = self.avg_pooling(inputs)
    elif self.pooling == 'max':
      x = self.max_pooling(inputs)

    if self.classes == 1:
      x = self.sigmoid_fc(x)
    else:
      x = self.softmax_fc(x)
      
    return x

class depthwise_separable_convolution(tf.keras.layers.Layer):
  def __init__(self, filter_num, stride, padd="same",use_bias=False):
    super(depthwise_separable_convolution, self).__init__()
    #dw
    self.dw_conv_1 = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                                strides=stride,
                                                padding=padd,
                                                use_bias=use_bias)
    self.bn1 = tf.keras.layers.BatchNormalization()

    #pw
    self.conv_2 = tf.keras.layers.Conv2D(filters=filter_num,
                                        kernel_size=(1, 1),
                                        strides=1,
                                        padding="same",
                                        use_bias=use_bias)
    self.bn2 = tf.keras.layers.BatchNormalization()

  def call(self, inputs):
    x = self.dw_conv_1(inputs)
    x = self.bn1(x)
    x = tf.nn.relu(x)

    x = self.conv_2(x)
    x = self.bn2(x)
    x = tf.nn.relu(x)
    
    return x
    

class MobileNet(tf.keras.Model):
  def __init__(self, include_top=True, classes=1000):
    super(MobileNet, self).__init__()

    self.include_top = include_top
    self.classes = classes

    self.conv_1 = tf.keras.layers.Conv2D(filters=32,
                                        kernel_size=(3, 3),
                                        strides=2,
                                        padding="same")
    #tf.keras.layers.SeparableConv2D구현(같다고는 못함 - 필요하면 직접 구현.)
    #MobileNet_V1 paper에서 averagepooling(pool_size=(7, 7), strides=1) 사용. 여기서는 글로벌평균 사용.
    self.dw_separable_block_1 = depthwise_separable_convolution(filter_num=64,
                                                                stride=1,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_2 = depthwise_separable_convolution(filter_num=128,
                                                                stride=2,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_3 = depthwise_separable_convolution(filter_num=128,
                                                                stride=1,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_4 = depthwise_separable_convolution(filter_num=256,
                                                                stride=2,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_5 = depthwise_separable_convolution(filter_num=256,
                                                                stride=1,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_6 = depthwise_separable_convolution(filter_num=512,
                                                                stride=2,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_7 = depthwise_separable_convolution(filter_num=512,
                                                                stride=1,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_8 = depthwise_separable_convolution(filter_num=1204,
                                                                stride=2,
                                                                padd="same",
                                                                use_bias=True)
    
    self.dw_separable_block_9 = depthwise_separable_convolution(filter_num=1024,
                                                                stride=2,
                                                                padd="same",
                                                                use_bias=True)
    self.fc_1 = MobileNet_classification(pooling='avg', 
                                         classes=1)
  def call(self, inputs):
    x = self.conv_1(inputs)
    x = self.dw_separable_block_1(x)
    x = self.dw_separable_block_2(x)
    x = self.dw_separable_block_3(x)
    x = self.dw_separable_block_4(x)
    x = self.dw_separable_block_5(x)
    x = self.dw_separable_block_6(x)
    for _ in range(5):
      x = self.dw_separable_block_7(x)
    x = self.dw_separable_block_8(x)
    x = self.dw_separable_block_9(x)
    if self.include_top:
      x = self.fc_1(x)
    return x

In [0]:
def MobileNet_V1():
  return MobileNet(include_top=True, classes=1)

In [0]:
MobileNet_v1 = MobileNet_V1()

In [0]:
def print_model_summary(network):
    network.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, 3))
    network.summary()

print_model_summary(network=MobileNet_v1)

Model: "mobile_net_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_100 (Conv2D)          multiple                  896       
_________________________________________________________________
depthwise_separable_convolut multiple                  2816      
_________________________________________________________________
depthwise_separable_convolut multiple                  9728      
_________________________________________________________________
depthwise_separable_convolut multiple                  18816     
_________________________________________________________________
depthwise_separable_convolut multiple                  35840     
_________________________________________________________________
depthwise_separable_convolut multiple                  70400     
_________________________________________________________________
depthwise_separable_convolut multiple                